In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import math
import glob
import csv
import os
import time
import matplotlib.pyplot as plt
from skimage.exposure import histogram
from matplotlib.pyplot import bar
from skimage.feature import local_binary_pattern
from skimage import io ,filters,feature,transform

from sklearn.svm import SVC
from mpl_toolkits.mplot3d import Axes3D

from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics

In [3]:
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        plt.axis('off')
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()

In [4]:
def Preprocess(image):
     
    # Remove salt and pepper noise    
    # Remove noise 
    median = cv2.medianBlur(image,5)
    blur = cv2.GaussianBlur(image,(5,5),0)
   
    greyImg = image
    
    # Otsu's Binarization
    ret3,img = cv2.threshold(image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
   
    # Remove header and footer
    length, width = img.shape
    up, down, left, right = 0, length - 1, 0, width - 1

    minWidthOfLines = width/2
    contours,__ = cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    for i in contours:
        x, y, w, h = cv2.boundingRect(i)
        if w < minWidthOfLines:
            continue
        if y < length // 2:
            up = max(up, y + 15)
        else:
            down = min(down, y - 15)

    offset = 30
    left = left + offset
    right = right -offset
    noHeaderAndFooter = img[up:down + 1, left:right + 1]
    noHeaderAndFooter = np.asarray(noHeaderAndFooter)
    
    noHeaderAndFooterGrey = greyImg[up:down + 1, left:right + 1]
    noHeaderAndFooterGrey = np.asarray(noHeaderAndFooterGrey) 
   
    # To crop the image
    row, col = noHeaderAndFooter.shape
    tolerance = 5

    sumOfRows = np.sum(noHeaderAndFooter, axis = 1)
    rowIndices = np.where(sumOfRows< (col-tolerance)*255)
    up = np.min(rowIndices)
    down = np.max(rowIndices)

    sumOfColoumns = np.sum(noHeaderAndFooter, axis = 0)
    colIndices = np.where(sumOfColoumns< (row-tolerance)*255)
    left = np.min(colIndices)
    right = np.max(colIndices)

    binarized = noHeaderAndFooter[up:down + 1, left:right + 1]
    binarized = np.asarray(binarized)
    
    greyscale = noHeaderAndFooterGrey[up:down + 1, left:right + 1]
    greyscale = np.asarray(greyscale)
   
    # Segmentation of Lines
    rowIndicesShifted = np.roll(rowIndices, -1)
    rowIndicesShifted = rowIndicesShifted[0]

    transitionIndices = np.where(np.abs(rowIndices - rowIndicesShifted) > 10)
    transitionIndices = transitionIndices[1]

    rowIndices = rowIndices[0]

    downIndices= rowIndices[transitionIndices]

    transitionIndicesUp = np.insert(transitionIndices,0,-1)
    transitionIndicesUp = np.delete(transitionIndicesUp,-1)

    upIndices= rowIndices[transitionIndicesUp+1]
    
    segmentsBinarized = []
    segmentsGrey = []
    totalSize=0
    whiteSpaceTolerance=0.975
    for i in range(transitionIndices.shape[0]):
        currSegment= noHeaderAndFooterGrey[upIndices[i]:downIndices[i] + 1, left:right + 1]
        if((np.sum(currSegment))<(currSegment.shape[0]*currSegment.shape[1]*whiteSpaceTolerance*255)):
            segmentsBinarized.append(noHeaderAndFooter[upIndices[i]:downIndices[i] + 1, left:right + 1])
            segmentsGrey.append(currSegment)
          
    segmentsGrey=np.asarray(segmentsGrey)
    
    return greyscale, binarized, segmentsBinarized, segmentsGrey

In [5]:
def showHist(imgHist):
    bar(imgHist[1].astype(np.uint8), imgHist[0], width=0.8, align='center')
    return imgHist[0]

In [6]:
#  Feature Extraction LBP
def LBP(greyscale):
    lbp = local_binary_pattern(greyscale, 8, 3, method='uniform')
    n_bins =10 #256 
    imgHist = histogram(lbp, n_bins)
    
    return imgHist[0];

In [7]:
def get_slant_angle(img):
    edges = feature.canny(img, sigma=0.6)
    hspace, angles, distances=transform.hough_line(edges)
    accum, angles, dists = transform.hough_line_peaks(hspace, angles, distances)
    angle = np.rad2deg(np.median(angles))
    return angle

In [8]:
def extract_features(greyscale1, binarized1, segmentsBinarized1, segmentsGrey1):
    features=[]
    features.extend(LBP(segmentsGrey1))
#     features.append(get_slant_angle(segmentsGrey1))
    
    return features

In [9]:
def train(folder):
    trainTime=0
    feature_vector_all=[]
    direcs = glob.glob (folder+"/*")
    for direc in direcs:
        if os.path.isdir(direc):
            print(direc)
            files = glob.glob (direc+'/*')
            for file in files:
                #reading an imag
                print("-----------------------",file,"------------------------------")
                img = cv2.imread(file,0) 
                tempStart =time.time()

                #converting it to a gray image
                #binarization process
                
                greyscale1, binarized1, segmentsBinarized1, segmentsGrey1 = Preprocess(img)
                print(segmentsGrey1.shape)
                for i in range(segmentsGrey1.shape[0]):
                    feature_vector=[]
                    
                    feature_vector.append(int(direc[8]))
                    
                    features = extract_features(greyscale1, binarized1, segmentsBinarized1, segmentsGrey1[i])
                    feature_vector.extend(features)
                    feature_vector_all.append(feature_vector)
                    
                tempEnd =time.time()
                trainTime+= tempEnd-tempStart 

    return feature_vector_all,trainTime

In [10]:
def calculateDistance(x1, x2):

    distance =np.linalg.norm(x1-x2)
    return distance

# def KNN(test_point, training_features, labels, k): 
#     y= labels[ np.argsort(calculateDistance(test_point,training_features))[:k]]
#     l,h = np.unique(y, return_counts=True)
#     return l[np.argmax(h)]

#     distarr=[]
#     for i in range (training_features.shape[0]):
#         f=calculateDistance(training_features[i,:],test_point)
#         distarr.append(f)
        
        
#     sortedarr=np.sort(distarr)
    
#     classes=np.zeros(3)
    
#     for i in range(k):
#         result = np.where(distarr == sortedarr[i])
#         print(result)
#         for j in range(3):
#             if(labels[result[0]]==j+1):
#                 classes[j]+=1
            
            
#     classification=np.argmax(classes)

  
#     return classification+1




In [11]:
def KNN(test_point, training_features, y_train, k):
    class1=0
    class2=0
    class3=0
    dist=[]
    indexs=[]
    
    for i in range(training_features.shape[0]):
        dist.append(calculateDistance(test_point,training_features[i]))
        
    dist2=np.argsort(dist)

    for i in range(k):
        if(y_train[dist2[i]]==1):
            class1=class1+1
        elif(y_train[dist2[i]]==2):
            class2=class2+1
        else:
            class3=class3+1

    if(max(class1,class2,class3)==class1):
        classification=1
    elif(max(class1,class2,class3)==class2):
        classification=2
    else:
        classification=3
    return classification

In [12]:
def SVM(training_features, y_train):
    clf = SVC(kernel='poly')    #linear sigmoid
    clf.fit(training_features, y_train)  
    return clf

In [13]:
# def adaboost_classifier(Y_train, X_train, Y_test, X_test, predictions):

#     M = X_train.shape[0]
#     w = np.full((M,), (1/M))
    
#     classValue = []
#     alpha_t =np.zeros(len(predictions))
        
#     for i in range(len(predictions)): 

#         miss=np.array(1*(predictions[i]==Y_test))

#         err_t = (w*(1-miss)).sum()/(w.sum())
        
#         print((w*(1-miss)).sum())
#         print(w.sum())
#         print(err_t)

#         if(err_t==0):
#             alpha_t[i] = (1)
#             break
#         else if(err_t==1):
#             alpha_t[i]=np.min(w)
#         else:
#             alpha_t[i] = (np.log((1-err_t)/err_t) + np.log(2))

#         w=w*np.exp(alpha_t[i]*(1-miss))
#         w= w/np.linalg.norm(w)    # Normalize weights
        
        
#     for k in range(3):
#         value=0
#         for j in range(len(predictions)):
#             value+=alpha_t[j]*(predictions[j]==k+1)
#         classValue.append(value)
        
#     print("classValue") 
#     print(classValue)
#     return np.argmax(classValue)+1

In [14]:
def adaboost_classifier(Y_train, training_features,clf,k):

    M = training_features.shape[0]
    w = np.full((M,), (1/M))
    
    classValue = []
    knnPredicts = []
    svmPredicts = []
    
    alpha_t =np.zeros(M)
    
    for i in range(M):
        knnPredicts=(KNN(training_features[i], training_features, Y_train, k))
        svmPredicts=(clf.predict([training_features[i]]))
      
    predictions = [knnPredicts,svmPredicts]
    for i in range(len(predictions)):  
        
        miss=np.array(1*(predictions[i]==Y_train))

        err_t = (w*(1-miss)).sum()/(w.sum())
        
#         print((w*(1-miss)).sum())
#         print(w.sum())
#         print(err_t)

        if(err_t==0):
            alpha_t[i] = (1)
            break
        elif(err_t==1):
            alpha_t[i]=np.min(w)
        else:
            alpha_t[i] = (np.log((1-err_t)/err_t) + np.log(2))

        w=w*np.exp(alpha_t[i]*(1-miss))
        w= w/np.linalg.norm(w)    # Normalize weights
        
    return alpha_t  
        
def adaboost_predict (X_test, alpha_t, predictions): 
    classValue = []
    for k in range(3):
        value=0
        for j in range(len(predictions)):
            value+=alpha_t[j]*(predictions[j]==k+1)
        classValue.append(value)
        
    print("classValue") 
    print(classValue)
    return np.argmax(classValue)+1

In [15]:
def main(directory,true_values,l):
    training_data,trainTime = train(directory)

    training_data = np.asarray(training_data)

    labels = training_data[:,0]
    training_features = training_data[:,1:]
    
    test_images = sorted(glob.glob(directory+'/*.png'))
    
#     true_values.append(int(str(os.path.basename(test_images[0])).split(".")[0]))
#     f3 = open(directory+"/ids.txt", "r")
    
    f3 = open(directory+'/ids.txt')
    ids = f3.readlines()
    ids=np.asarray(ids,dtype=int)
    xtest=ids[-1]
    ytest=np.where(ids==xtest)[0][0]+1 
    true_values.append(ytest)
    f3.close()
    
    k = 5
    knn_prediction = []
    SVM_prediction=[]
    ADA_prediction=[]
    ADA2_prediction=[]
    
    totalTime=[]

    f = open("time.txt", "a")
    f2= open("results.txt","a")
    
    clf= SVM(training_features,labels)
    
    alpha_t = adaboost_classifier(labels, training_features,clf, k)    # ADA2
    
    ada = AdaBoostClassifier(n_estimators=50, learning_rate=1)    #ADA
    ADA_model = ada.fit(training_features, labels)
    
    for i in range(len(test_images)):
    
        img_original = cv2.imread(test_images[i],0)
        start = time.time()
        greyscale1, binarized1, segmentsBinarized1, segmentsGrey1 = Preprocess(img_original)
        maxOccurSegment=[]
        maxOccurSegmentSVM=[]
        maxOccurSegmentADA=[]
        maxOccurSegmentADA2=[]
        print("Actual class :", true_values[l], " of file ", test_images[i])
        print("---------------------------------------")
        for j in range(segmentsGrey1.shape[0]):
            test_point = extract_features(greyscale1, binarized1, segmentsBinarized1, segmentsGrey1[j])

            maxOccurSegment.append(int(KNN(test_point, training_features, labels, k)))
            maxOccurSegmentSVM.append(int(clf.predict([test_point])[0]))  #SVM
            maxOccurSegmentADA.append(int(ADA_model.predict([test_point])[0]))  #ADA
            
            predictions = [maxOccurSegment[j], maxOccurSegmentSVM[j]]
#             maxOccurSegmentADA2.append(adaboost_classifier(labels, training_features, true_values[l], test_point, predictions)) #ADA2 #
            maxOccurSegmentADA2.append(adaboost_predict(test_point, alpha_t, predictions)) #ADA2 #


        maxOccurSegment=np.asarray(maxOccurSegment)
        maxOccurSegmentSVM=np.asarray(maxOccurSegmentSVM)
        maxOccurSegmentADA=np.asarray(maxOccurSegmentADA)
        maxOccurSegmentADA2=np.asarray(maxOccurSegmentADA2)

        print(maxOccurSegment)
        print(maxOccurSegmentSVM)
        print(maxOccurSegmentADA)
        print(maxOccurSegmentADA2)
        
        knn_prediction.append(np.bincount(maxOccurSegment).argmax())
        SVM_prediction.append(np.bincount(maxOccurSegmentSVM).argmax())
        ADA_prediction.append(np.bincount(maxOccurSegmentADA).argmax())
        ADA2_prediction.append(np.bincount(maxOccurSegmentADA2).argmax())
        
        end = time.time()
        predictionTime= end-start
        totalTime.append(trainTime+ predictionTime)

        f.write(str(round(totalTime[i],2)))
        f.write("\n")

        f2.write(str(knn_prediction[i]))
        f2.write("\n")


        
        print("knn_prediction class :", knn_prediction[i])
        print("SVM_prediction class :", SVM_prediction[i])
        print("ADA_prediction class :", ADA_prediction[i])
        print("ADA2_prediction class :", ADA2_prediction[i])
        # ------------------------------------------------------------------------------------------------------

    f.write("------------------------------------")
    totalTime=np.asarray(totalTime)
    f.write(str(np.mean(totalTime)))
    f.close()
    f2.close()

    
    return knn_prediction, SVM_prediction, ADA_prediction, ADA2_prediction

In [16]:
direcs = glob.glob ("data/*")
true_values = []
knn_prediction = []
SVM_prediction=[]
ADA_prediction=[]
ADA2_prediction=[]

for l in range(len(direcs)):
    knn_predict, SVM_predict, ADA_predict, ADA2_predict  = main(direcs[l],true_values,l)
    knn_prediction.append(knn_predict[0])
    SVM_prediction.append(SVM_predict[0])
    ADA_prediction.append(ADA_predict[0])
    ADA2_prediction.append(ADA2_predict[0])

data\00\1
----------------------- data\00\1\b06-042.png ------------------------------


C:\Users\nadin\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


(11,)
----------------------- data\00\1\b06-068.png ------------------------------
(11,)
data\00\2
----------------------- data\00\2\a02-037.png ------------------------------
(10,)
----------------------- data\00\2\a02-090.png ------------------------------
(7,)
data\00\3
----------------------- data\00\3\b04-081.png ------------------------------
(1, 1262, 1912)
----------------------- data\00\3\b04-103.png ------------------------------
(1, 1268, 1913)


C:\Users\nadin\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Actual class : 3  of file  data\00\b04-096.png
---------------------------------------
classValue
[0.8397506547518203, 0.0, -1.890850371872284]
classValue
[-1.0510997171204637, 0.0, 0.0]
[1 1]
[3 1]
[3 1]
[1 2]
knn_prediction class : 1
SVM_prediction class : 1
ADA_prediction class : 1
ADA2_prediction class : 1
data\01\1
----------------------- data\01\1\g06-011k.png ------------------------------


...

Actual class : 1  of file  data\06\1.png
---------------------------------------
classValue
[0.32542240043462645, 0.0, 0.0]
classValue
[-1.7763568394002505e-15, 0.0, 0.3254224004346282]
classValue
[0.32542240043462645, 0.0, 0.0]
classValue
[0.32542240043462645, 0.0, 0.0]
classValue
[-1.7763568394002505e-15, 0.0, 0.3254224004346282]
classValue
[-1.7763568394002505e-15, 0.0, 0.3254224004346282]
classValue
[0.32542240043462645, 0.0, 0.0]
classValue
[0.32542240043462645, 0.0, 0.0]
[1 3 1 1 3 3 1 1]
[1 1 1 1 1 1 1 1]
[1 3 3 1 3 3 1 1]
[1 3 1 1 3 3 1 1]
knn_prediction class : 1
SVM_prediction class : 1
ADA_prediction class : 1
ADA2_prediction class : 1
data\07\1
----------------------- data\07\1\a01-000u.png ------------------------------


C:\Users\nadin\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


(7,)
----------------------- data\07\1\a01-003u.png ------------------------------
(11,)
data\07\2
----------------------- data\07\2\r06-130.png ------------------------------
(10,)
----------------------- data\07\2\r06-137.png ------------------------------
(11,)
data\07\3
----------------------- data\07\3\g06-011n.png ------------------------------
(11,)
----------------------- data\07\3\g06-018n.png ------------------------------
(10,)


C:\Users\nadin\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Actual class : 1  of file  data\07\1.png
---------------------------------------
classValue
[0.07410797215372145, 0.0, 0.0]
classValue
[0.07410797215372145, 0.0, 0.0]
classValue
[0.07410797215372145, 0.0, 0.0]
classValue
[0.07410797215372145, 0.0, 0.0]
classValue
[0.07410797215372145, 0.0, 0.0]
classValue
[0.07410797215372145, 0.0, 0.0]
classValue
[0.07410797215372145, 0.0, 0.0]
classValue
[0.07410797215372145, 0.0, 0.0]
classValue
[0.07410797215372145, 0.0, 0.0]
classValue
[0.07410797215372145, 0.0, 0.0]
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 2 1 1 1 2]
[1 1 1 1 1 1 1 1 1 1]
knn_prediction class : 1
SVM_prediction class : 1
ADA_prediction class : 1
ADA2_prediction class : 1


In [17]:
total_predictions =  len(direcs)
correct_knn=0
correct_SVM=0
correct_ADA=0
correct_ADA2=0

print(true_values)
print(knn_prediction)
print(SVM_prediction)
print(ADA_prediction)
print(ADA2_prediction)
for i in range(total_predictions):
    if(true_values[i]==knn_prediction[i]):
        correct_knn=correct_knn+1
    if(true_values[i]==SVM_prediction[i]):
        correct_SVM=correct_SVM+1
    if(true_values[i]==ADA_prediction[i]):
        correct_ADA=correct_ADA+1
    if(true_values[i]==ADA2_prediction[i]):
        correct_ADA2=correct_ADA2+1   


accuracy_knn = (correct_knn/total_predictions)*100
accuracy_SVM = (correct_SVM/total_predictions)*100
accuracy_ADA = (correct_ADA/total_predictions)*100
accuracy_ADA2 = (correct_ADA2/total_predictions)*100
# ------------------------------------------------------------------------------------------------------

print("K-Nearest Neighbour Classifier Accuracy: ", accuracy_knn, "%")
print("SVM Classifier Accuracy: ", accuracy_SVM, "%")
print("ADA Classifier Accuracy: ", accuracy_ADA, "%")
print("ADA2 Classifier Accuracy: ", accuracy_ADA2, "%")

[3, 2, 1, 1, 3, 2, 1, 1]
[1, 2, 1, 1, 3, 2, 1, 1]
[1, 2, 2, 1, 3, 2, 1, 1]
[1, 2, 1, 1, 3, 2, 1, 1]
[1, 2, 1, 1, 3, 2, 1, 1]
K-Nearest Neighbour Classifier Accuracy:  87.5 %
SVM Classifier Accuracy:  75.0 %
ADA Classifier Accuracy:  87.5 %
ADA2 Classifier Accuracy:  87.5 %


In [18]:
# def adaboost_classifier(Y_train, X_train, Y_test, X_test, predictions):

#     M = X_train.shape[0]
#     w = np.full((M,), (1/M))
    
#     classValue = []
        
#     for k in range(3):
#         for i in range(T): 

#             miss=np.array(1*(predictions[i]==Y_train))

#             err_t = (w*(1-miss)).sum()/(w.sum())

#             alpha_t = np.log((1-err_t)/err_t) + np.log(2)

#             w=w*np.exp(alpha_t*(1-miss))
#             w= w/np.linalg.norm(w)    # Normalize weights
    
#             classValue.append(alpha_t*(predictions[i]==k+1))
        
#     return np.argmax(classValue)+1
# Preprocess    
# data\00\2\a02-090.png    
